In [1]:
import pandas as pd
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
study_name = COMBINE_harmonizer.STUDY_LH
sheet_name = COMBINE_harmonizer.SHEET_FOLLOW_UP

root_dir = '..'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name)

In [6]:
dict_columns = list(df_dict[study_name].unique())

## 03. LH

In [7]:
the_dir = cfg.config[f'{study_name}_follow_up_dir']

In [8]:
filenames = os.listdir(the_dir)
filenames.sort()

full_filenames = [os.sep.join([the_dir, each_filename]) for each_filename in filenames]

In [9]:
analysis_dir = cfg.config[f'{study_name}_analysis_dir']

In [10]:
analysis_filenames = os.listdir(analysis_dir)
analysis_filenames.sort()

full_analysis_filenames = [os.sep.join([analysis_dir, each_filename]) for each_filename in analysis_filenames]

In [11]:
all_filenames = filenames + analysis_filenames
full_all_filenames = full_filenames + full_analysis_filenames
dfs = [pd.read_csv(each_filename) for each_filename in full_all_filenames]

### 03.1. check columns in each file

In [12]:
for idx, each_df in enumerate(dfs):
    print(f'({idx}/{len(dfs)}) filename: {all_filenames[idx]} columns: {list(each_df.columns)}')

(0/12) filename: lf01.csv columns: ['LHFOLNUM', 'SITENM', 'LAST', 'FIRST', 'HTHRM_ID', 'PROTID02', 'PROTID03', 'PROTID04', 'PROTID05', 'PROTID06', 'PROTID07', 'PROTID08', 'PROTID09', 'PROTID10', 'PROTID11', 'PROTID12', 'PROTID13', 'PROTID14', 'PROTID15', 'PROTID16', 'PROTID17', 'PROTID18', 'PROTID19', 'PROTID20', 'REC_CMP', 'CENTER', 'BIRTHDT', 'VISITDT', 'BIRTHNM', 'PREVCNTR', 'CMP_DATE', 'CRT_DATE']
(1/12) filename: lf03.csv columns: ['LHFOLNUM', 'CFSUPER', 'NF3MONEY', 'CFWKPCAR', 'CFWKOCAR', 'CFSCCARE', 'CFSCADUL', 'CFOTHLAN', 'CFSECLAN', 'CFOTHSEC', 'CFNDNRSV', 'CFNDNRSH', 'CFNDOTPT', 'CFNDSPCH', 'CFNDEALY', 'CFNDSOCL', 'CFNDCLIN', 'CFNDPUL', 'CFNDOPH', 'CFNDGAS', 'CFNDAUD', 'CFNDNEU', 'CFNDOTH', 'CFCVOSP', 'CFNDEVAL', 'CFNDPFU', 'CFREGDR', 'CFCCFAC', 'CFCAROT', 'CFTRDYCR', 'CFSPDYCR', 'CFHMDYCR', 'CFBBYSIT', 'CFINITS', 'REC_CMP', 'CENTER', 'CFVISDT', 'CFBRTHDT', 'CFCHRAGE', 'CFADJAGE', 'CFCARE', 'CFOCAR', 'CFMARITL', 'CFLIVING', 'CFPEOPLE', 'CFGRPCAR', 'CFGROCAR', 'CFINCOME', 'CFI

### 03.2 all columns

In [13]:
all_columns = []
for idx, each_df in enumerate(dfs):
    all_columns += list(map(lambda x: {'column': x, 'idx': idx, 'filename': all_filenames[idx]}, list(each_df.columns)))
df_columns = pd.DataFrame(all_columns)
df_columns['column:filename'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}", axis=1)
df_columns['column:filename:idx'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}:{x['idx']}", axis=1)
df_columns.sort_values(by=['filename', 'column'], inplace=True)

In [14]:
df_columns

,column,idx,filename,column:filename,column:filename:idx
439,CENTER,10,analysis.csv,CENTER:analysis.csv,CENTER:analysis.csv:10
438,HTHRM_ID,10,analysis.csv,HTHRM_ID:analysis.csv,HTHRM_ID:analysis.csv:10
452,LHFOLNUM,10,analysis.csv,LHFOLNUM:analysis.csv,LHFOLNUM:analysis.csv:10
461,MRI_AGED,10,analysis.csv,MRI_AGED:analysis.csv,MRI_AGED:analysis.csv:10
459,MRI_DATE,10,analysis.csv,MRI_DATE:analysis.csv,MRI_DATE:analysis.csv:10
...,...,...,...,...,...
491,hear_imp,11,mrilh.csv,hear_imp:mrilh.csv,hear_imp:mrilh.csv:11
490,hearing,11,mrilh.csv,hearing:mrilh.csv,hearing:mrilh.csv:11
498,hosp_die,11,mrilh.csv,hosp_die:mrilh.csv,hosp_die:mrilh.csv:11
494,modsevcp,11,mrilh.csv,modsevcp:mrilh.csv,modsevcp:mrilh.csv:11


In [15]:
df_columns_count = df_columns.groupby(['column']).agg(_count=('column', 'count')).sort_values(by=['_count'], ascending=False)
df_columns_count.head(20)

,_count
column,
CENTER,12
LHFOLNUM,12
CRT_DATE,10
CMP_DATE,10
REC_CMP,10
HTHRM_ID,3
seiz_fu,2
MRI_NOTDONE,2
MRI_INFARR_WG,2


### 03.3 check LHFOLNUM

In [16]:
is_LHFOLNUM = df_columns['column'] == 'LHFOLNUM'
df_columns[is_LHFOLNUM]

,column,idx,filename,column:filename,column:filename:idx
452,LHFOLNUM,10,analysis.csv,LHFOLNUM:analysis.csv,LHFOLNUM:analysis.csv:10
0,LHFOLNUM,0,lf01.csv,LHFOLNUM:lf01.csv,LHFOLNUM:lf01.csv:0
32,LHFOLNUM,1,lf03.csv,LHFOLNUM:lf03.csv,LHFOLNUM:lf03.csv:1
112,LHFOLNUM,2,lf04.csv,LHFOLNUM:lf04.csv,LHFOLNUM:lf04.csv:2
183,LHFOLNUM,3,lf04a.csv,LHFOLNUM:lf04a.csv,LHFOLNUM:lf04a.csv:3
193,LHFOLNUM,4,lf04ar.csv,LHFOLNUM:lf04ar.csv,LHFOLNUM:lf04ar.csv:4
202,LHFOLNUM,5,lf05.csv,LHFOLNUM:lf05.csv,LHFOLNUM:lf05.csv:5
300,LHFOLNUM,6,lf09a.csv,LHFOLNUM:lf09a.csv,LHFOLNUM:lf09a.csv:6
346,LHFOLNUM,7,lf10.csv,LHFOLNUM:lf10.csv,LHFOLNUM:lf10.csv:7
359,LHFOLNUM,8,lf11.csv,LHFOLNUM:lf11.csv,LHFOLNUM:lf11.csv:8


##### column not in dict

In [17]:
is_not_in_dict = df_columns['column'].isin(dict_columns) == False
df_columns_not_in_dict = df_columns[is_not_in_dict]
df_columns_not_in_dict

,column,idx,filename,column:filename,column:filename:idx
461,MRI_AGED,10,analysis.csv,MRI_AGED:analysis.csv,MRI_AGED:analysis.csv:10
459,MRI_DATE,10,analysis.csv,MRI_DATE:analysis.csv,MRI_DATE:analysis.csv:10
477,MRI_INFARL,10,analysis.csv,MRI_INFARL:analysis.csv,MRI_INFARL:analysis.csv:10
479,MRI_INFARL_NR,10,analysis.csv,MRI_INFARL_NR:analysis.csv,MRI_INFARL_NR:analysis.csv:10
478,MRI_INFARL_PB,10,analysis.csv,MRI_INFARL_PB:analysis.csv,MRI_INFARL_PB:analysis.csv:10
...,...,...,...,...,...
508,MRI_TIME,11,mrilh.csv,MRI_TIME:mrilh.csv,MRI_TIME:mrilh.csv:11
510,MRI_UNREAD,11,mrilh.csv,MRI_UNREAD:mrilh.csv,MRI_UNREAD:mrilh.csv:11
499,any_antic,11,mrilh.csv,any_antic:mrilh.csv,any_antic:mrilh.csv:11
501,any_seiz,11,mrilh.csv,any_seiz:mrilh.csv,any_seiz:mrilh.csv:11


In [18]:
columns_not_in_dict = list(df_columns_not_in_dict['column:filename:idx'].unique())
# columns_not_in_dict.sort()
columns_not_in_dict

['MRI_AGED:analysis.csv:10',
 'MRI_DATE:analysis.csv:10',
 'MRI_INFARL:analysis.csv:10',
 'MRI_INFARL_NR:analysis.csv:10',
 'MRI_INFARL_PB:analysis.csv:10',
 'MRI_INFARL_WG:analysis.csv:10',
 'MRI_INFARR:analysis.csv:10',
 'MRI_INFARR_NR:analysis.csv:10',
 'MRI_INFARR_PB:analysis.csv:10',
 'MRI_INFARR_WG:analysis.csv:10',
 'MRI_NOTDONE:analysis.csv:10',
 'MRI_PATTERN:analysis.csv:10',
 'MRI_PATTERN_LC:analysis.csv:10',
 'MRI_PATTERN_NR:analysis.csv:10',
 'MRI_PATTERN_PB:analysis.csv:10',
 'MRI_PATTERN_WG:analysis.csv:10',
 'MRI_PCLASS:analysis.csv:10',
 'MRI_PCLASS_LC:analysis.csv:10',
 'MRI_PCLASS_NR:analysis.csv:10',
 'MRI_PCLASS_PB:analysis.csv:10',
 'MRI_PCLASS_WG:analysis.csv:10',
 'MRI_RDATE_LC:analysis.csv:10',
 'MRI_RDATE_NR:analysis.csv:10',
 'MRI_RDATE_PB:analysis.csv:10',
 'MRI_RDATE_WG:analysis.csv:10',
 'MRI_TIME:analysis.csv:10',
 'MRI_UNREAD:analysis.csv:10',
 'any_antic:analysis.csv:10',
 'any_seiz:analysis.csv:10',
 'fcenter:analysis.csv:10',
 'CMP_DATE:lf01.csv:0',
 '

#### dict not in column

In [19]:
is_dict_not_in_columns = (df_dict[study_name].isin(df_columns['column']) == False) & (df_dict[study_name].isnull() == False)
df_dict_not_in_columns = df_dict[is_dict_not_in_columns]
df_dict_not_in_columns

,Category,Subcategory,Standardized_VariableNames_Dictionary,type,Variable_Description,#studies w/ this var,redcap,comment,lower_var,var_eq_redcap,connect_redcap,LH,OC


## 04. show column values

In [20]:
# XXX no show due to data privacy concern.

'''
for idx, each_df in enumerate(dfs):
    for idx2, column in enumerate(each_df.columns):
        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')
'''

"\nfor idx, each_df in enumerate(dfs):\n    for idx2, column in enumerate(each_df.columns):\n        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')\n"